### Southbank location using all violation instances

This workbook steps through investigation into one area within the data set. Due to the sensor data file being so large (over 5gb) it is not possible to view all parking violation instances across the city. I've chosen to drill down on Southbank as this area had the top car parking bays with the highest overall parking violation instances in 2017. 
The difference between this approach and the all_violations aggregate approach is that information over time can be analysed.
The same data calling, cleaning, merging steps apply as the all_violations notebook

In [ ]:
# make sure to install these packages before running:
# pip install pandas -- already installed

In [1]:
#pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.melbourne.vic.gov.au", "RG1MLdEhMl8N8uP4WMDWLJ3Yz")

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.melbourne.vic.gov.au,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

# Full year 2017 parking events for Southbank, violations, arrival date, street, row is a parking instance
results = client.get("u9sa-j86i", area='Southbank', in_violation=True, limit=110000)

# All on-street parking bays: lat, long of parking bays
results2 = client.get("wuf8-susg", limit=30000)

# All parking bay sensors --> not used, this is the live parking information
results3 = client.get("vh2v-4nfs", limit=5000)

# All parking restrictions: extended info for restrictions
results4 = client.get("ntht-5rk7", limit=4200)

# Convert to pandas DataFrame
sensor_df = pd.DataFrame.from_records(results)

# Convert to pandas DataFrame
location_df = pd.DataFrame.from_records(results2)

# Convert to pandas DataFrame
park_sens_df = pd.DataFrame.from_records(results3)

# # Convert to pandas DataFrame
restriction_df = pd.DataFrame.from_records(results4)

In [ ]:
# response = requests.get('https://data.melbourne.vic.gov.au/resource/ntht-5rk7.json')
# status_code = response.status_code
# data_parking_res = response.json()

### Check and clean sensor_df

In [2]:
sensor_df.head(2)

,deviceid,arrivaltime,departuretime,durationseconds,streetmarker,sign,area,streetid,streetname,betweenstreet1,betweenstreet2,side_of_street,in_violation,vehicle_present
0,21668,2017-11-13T13:08:02.000,2017-11-13T16:14:31.000,11189,8642E,3P TKT A M-F 7:30-18:30,Southbank,828,KAVANAGH STREET,BALSTON STREET,POWER STREET,2,True,True
1,21416,2017-02-10T07:30:00.000,2017-02-10T10:56:47.000,12407,8350E,3P TKT A M-F 7:30-18:30,Southbank,591,DODDS STREET,SOUTHBANK BOULEVARD,GRANT STREET,2,True,True


In [3]:
sensor_df.shape
# this row number matches the website visualisation 

(105151, 14)

In [9]:
sensor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105151 entries, 0 to 105150
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   deviceid         105151 non-null  object
 1   arrivaltime      105151 non-null  object
 2   departuretime    105151 non-null  object
 3   durationseconds  105151 non-null  object
 4   streetmarker     105151 non-null  object
 5   sign             105148 non-null  object
 6   area             105151 non-null  object
 7   streetid         105151 non-null  object
 8   streetname       105151 non-null  object
 9   betweenstreet1   105151 non-null  object
 10  betweenstreet2   105151 non-null  object
 11  side_of_street   105151 non-null  object
 12  in_violation     105151 non-null  object
 13  vehicle_present  105151 non-null  object
dtypes: object(14)
memory usage: 11.2+ MB


In [43]:
sensor_df.isnull().sum()
# only 3 nulls in sign column (won't use this column)

deviceid           0
arrivaltime        0
departuretime      0
durationseconds    0
marker_id          0
sign               3
area               0
streetid           0
streetname         0
betweenstreet1     0
betweenstreet2     0
side_of_street     0
in_violation       0
vehicle_present    0
dtype: int64

### Check/clean location_df

In [15]:
location_df.shape

(24746, 8)

In [16]:
location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24746 entries, 0 to 24745
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   the_geom     24746 non-null  object
 1   marker_id    6015 non-null   object
 2   bay_id       24746 non-null  object
 3   last_edit    24746 non-null  object
 4   rd_seg_id    23600 non-null  object
 5   rd_seg_dsc   23600 non-null  object
 6   meter_id     533 non-null    object
 7   coordinates  24746 non-null  object
dtypes: object(8)
memory usage: 1.5+ MB


In [5]:
location_df = location_df.dropna(subset = ['marker_id'])

In [18]:
location_df.marker_id.value_counts().sort_values(ascending = False)

3776N     4
15208E    4
4559W     2
9083S     2
13436E    2
         ..
C814      1
6581S     1
15502N    1
3407S     1
2853S     1
Name: marker_id, Length: 6001, dtype: int64

In [6]:
location_df = location_df.drop_duplicates(subset = 'marker_id')

In [20]:
location_df.marker_id.value_counts().sort_values(ascending = False)
#shows duplicates removed

7062N     1
12568N    1
C592      1
1526E     1
13828N    1
         ..
C814      1
6581S     1
15502N    1
3407S     1
4869W     1
Name: marker_id, Length: 6001, dtype: int64

In [21]:
location_df.shape

(6001, 8)

In [22]:
location_df.head(2)

,the_geom,marker_id,bay_id,last_edit,rd_seg_id,rd_seg_dsc,meter_id,coordinates
0,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",20268E,17200,20201223122254,22277,St Leonards Court from Anderson Street,NaN,"[144.985530949946, -37.829757851313]"
9,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",4902E,14322,20201208141232,21630,Queen Street between Franklin Street and Therr...,NaN,"[144.958094708823, -37.808001781862]"


#### Looking at the 'the_geom'

In [23]:
location_df['the_geom'][0]

{'type': 'MultiPolygon',
 'coordinates': [[[[144.985530949946, -37.829757851313],
    [144.985512852395, -37.829763753466],
    [144.985510384116, -37.829757713584],
    [144.985508481348, -37.829751561613],
    [144.985507310993, -37.82974529717],
    [144.985501612155, -37.829713747685],
    [144.985521539055, -37.829711225609],
    [144.985523760892, -37.829725114714],
    [144.985525367663, -37.829738725646],
    [144.985529858079, -37.829756054609],
    [144.985530949946, -37.829757851313]]]]}

#### Extract the first set of coordinates from the_geom and add as lat/long columns

The car park bay location is given as a spatial polygon. An approach to extract one lat/long coordinate per parking bay is to get the first set of coordinates and use that as the location, which is shown below.

In [7]:
location_df['coordinates'] = location_df.the_geom.apply(lambda x: x['coordinates'][0][0][0])
# get first item in the nested list --> assign as new column called coordinates

In [8]:
location_df[['long','lat']] = pd.DataFrame(location_df['coordinates'].tolist(), index = location_df.index)

In [26]:
location_df.head(2)

,the_geom,marker_id,bay_id,last_edit,rd_seg_id,rd_seg_dsc,meter_id,coordinates,long,lat
0,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",20268E,17200,20201223122254,22277,St Leonards Court from Anderson Street,NaN,"[144.985530949946, -37.829757851313]",144.985531,-37.829758
9,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",4902E,14322,20201208141232,21630,Queen Street between Franklin Street and Therr...,NaN,"[144.958094708823, -37.808001781862]",144.958095,-37.808002


#### Merge on streetmarker/marker_id

In [9]:
sensor_df.rename(columns = {'streetmarker':'marker_id'}, inplace = True)
# Rename the column name so it's easy to match on one column: 'marker_id'

In [29]:
sensor_df.columns

Index(['deviceid', 'arrivaltime', 'departuretime', 'durationseconds',
       'marker_id', 'sign', 'area', 'streetid', 'streetname', 'betweenstreet1',
       'betweenstreet2', 'side_of_street', 'in_violation', 'vehicle_present'],
      dtype='object')

In [30]:
location_df.columns

Index(['the_geom', 'marker_id', 'bay_id', 'last_edit', 'rd_seg_id',
       'rd_seg_dsc', 'meter_id', 'coordinates', 'long', 'lat'],
      dtype='object')

In [36]:
sensor_df['marker_id'].nunique()

497

In [32]:
# this function checks to see unique values in the matching column across both datasets. It helps to ensure
# you don't lose/gain any rows through the merge step
def check_if_match(df_1, df_2, col_name_df1, col_name_df2):
    unique_to_df_1 = [i for i in df_1[col_name_df1].unique() if i not in df_2[col_name_df2].unique()]
    unique_to_df_2 = [i for i in df_2[col_name_df2].unique() if i not in df_1[col_name_df1].unique()]
    common_to_df_1_and_df_2 = [i for i in df_2[col_name_df2].unique() if i in df_1[col_name_df1].unique()]
    print('values unique to df_1', str(len(unique_to_df_1)))
    print('values unique to df_2', str(len(unique_to_df_2)))
    print('values common to df_1 and df_2', str(len(common_to_df_1_and_df_2)))
    return unique_to_df_1, unique_to_df_2, common_to_df_1_and_df_2

In [33]:
# this runs the function: note it can take a while to run if large dataset
result_unique_df_1, result_unique_df_2, result_common_df_1_df_2 = check_if_match(df_1 = sensor_df, df_2 = location_df, col_name_df1 = 'marker_id', col_name_df2 = 'marker_id')

values unique to df_1 62
values unique to df_2 5566
values common to df_1 and df_2 435


In [34]:
# result_common_df_1_df_2 
# to see list of marker id's that will be used in the join

In [35]:
# result_unique_df_1  
# this shows how many marker id's in the sensor df don't have a matching lat/long and therefore will not be included
# in further analysis

In [34]:
# result_unique_df_2
# these are unique to the location_df: makes sense that this list is large due to only one area selected

In [37]:
location_df.marker_id.nunique()

6001

In [10]:
sensor_merge_location = pd.merge(sensor_df, location_df, on="marker_id")

In [11]:
sensor_merge_location.marker_id.nunique()
# this number matches our marker id's in common between the df's

435

In [40]:
sensor_df.shape

(105151, 14)

In [41]:
sensor_merge_location.shape
# lost some rows where no markerid and therefore no lat/long info in location_df

(89778, 23)

In [12]:
sensor_merge_location.head(2)

,deviceid,arrivaltime,departuretime,durationseconds,marker_id,sign,area,streetid,streetname,betweenstreet1,...,vehicle_present,the_geom,bay_id,last_edit,rd_seg_id,rd_seg_dsc,meter_id,coordinates,long,lat
0,21668,2017-11-13T13:08:02.000,2017-11-13T16:14:31.000,11189,8642E,3P TKT A M-F 7:30-18:30,Southbank,828,KAVANAGH STREET,BALSTON STREET,...,True,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",22717,20201208142107,22051,Kavanagh Street between Power Street and Balst...,NaN,"[144.962619935411, -37.825585019738]",144.96262,-37.825585
1,21668,2017-06-14T10:15:43.000,2017-06-14T13:53:04.000,13041,8642E,3P TKT A M-F 7:30-18:30,Southbank,828,KAVANAGH STREET,BALSTON STREET,...,True,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",22717,20201208142107,22051,Kavanagh Street between Power Street and Balst...,NaN,"[144.962619935411, -37.825585019738]",144.96262,-37.825585


In [14]:
Southbank_violations_interesting_columns = sensor_merge_location[['deviceid','arrivaltime','departuretime','durationseconds','marker_id','area','streetname','bay_id','long','lat']]

In [15]:
Southbank_violations_interesting_columns.head(5)

,deviceid,arrivaltime,departuretime,durationseconds,marker_id,area,streetname,bay_id,long,lat
0,21668,2017-11-13T13:08:02.000,2017-11-13T16:14:31.000,11189,8642E,Southbank,KAVANAGH STREET,22717,144.96262,-37.825585
1,21668,2017-06-14T10:15:43.000,2017-06-14T13:53:04.000,13041,8642E,Southbank,KAVANAGH STREET,22717,144.96262,-37.825585
2,21668,2017-10-10T07:30:00.000,2017-10-10T18:30:00.000,39600,8642E,Southbank,KAVANAGH STREET,22717,144.96262,-37.825585
3,21668,2017-02-13T14:38:11.000,2017-02-13T18:30:00.000,13909,8642E,Southbank,KAVANAGH STREET,22717,144.96262,-37.825585
4,21668,2017-07-31T07:30:00.000,2017-07-31T10:52:35.000,12155,8642E,Southbank,KAVANAGH STREET,22717,144.96262,-37.825585


In [44]:
sensor_merge_location.to_csv('SouthbankViolations.csv')
#export to csv to view in Tableau